In [1]:
import json

In [9]:
with open('./data/train.json') as d:
    json_data = json.load(d)

In [12]:
json_data[0]

{'body': 'Every day, cubicle-dwellers get up from their desks and walk to their nearest windows in search of a strong enough signal to make or answer a cell-phone call. Cathy Zatloukal thinks her in-building wireless infrastructure company, MobileAccess Networks, can save them the trip.',
 'category': 4,
 'title': 'MobileAccess Networks Strengthens Signals for Indoor Use'}

In [35]:
def get_doc_terms(doc_info):
    body_terms = data['body'].split()
    title_terms = data['title'].split()
    all_terms = body_terms + title_terms
    return all_terms

In [38]:
vectors = {}
query = {}
docs_no = len(json_data)

for index, data in enumerate(json_data):

    index = str(index)
    vectors[index] = {
        'terms': {},
        'category': 0
    }
    
    all_terms = get_doc_terms(data)
    
    for term in all_terms:
        if term in vectors[index]['terms']:
            vectors[index]['terms'][term] += 1
        else:
            vectors[index]['terms'][term] = 1
    
    vectors[index]['category'] = data['category']

In [37]:
def cosine_similarity(q, v):
    cosine_similarity_value = 0 
    
    numerator = 0
    denominator = 1
    
    for term in q:
        if term in v:
            numerator += q[term] * v[ter]

def KNN(query_info, k = 1):
    
    query_terms = get_doc_terms(query_info)
    for term in query_terms:
        if term in query:
            query[term] += 1
        else:
            query[term] = 1
    
    